### Question list:
Should x4(categorical) be standarized?

Shall we find outliers and delete them?

Standarization-->Split, or the other way round?

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate,train_test_split,KFold,LeaveOneOut,cross_val_score,cross_val_predict
from matplotlib import pyplot as plt
from sklearn.linear_model import Ridge 
# from sklearn.model_selection import train_test_split 
# from sklearn.datasets import load_boston 
from sklearn.preprocessing import StandardScaler 

from scipy import stats

In [30]:
url = 'https://raw.githubusercontent.com/orangeczw/ML_team_Name/master/Task_1a/train.csv'
train = pd.read_csv(url, error_bad_lines=False)
y = train.iloc[:,1].copy()
X = train.iloc[:,2:].copy()

In [3]:
X.describe()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.593761,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.596783,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.647422,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [4]:
#See if there's NA value
X.isna().any()

x1     False
x2     False
x3     False
x4     False
x5     False
x6     False
x7     False
x8     False
x9     False
x10    False
x11    False
x12    False
x13    False
dtype: bool

#### Standardization + using cross_val_predict


In [21]:
# scaling the inputs 
scaler = StandardScaler() 
scaled_X = scaler.fit_transform(X) 

#Standarize by above steps or 'normalize = False'?
penalty = [0.01,0.1,1,10,100]
RMSE = []

for lmd in penalty:
    model = Ridge(alpha = lmd, normalize = False, tol = 0.001, solver ='auto', random_state = 42) #solver?random_state?
#     cv_results = cross_validate(model, scaled_X, y, cv=10)
    predictions = cross_val_predict(model, scaled_X, y, cv=10)
#     plt.scatter(y, predictions)
    error = np.sqrt((y-predictions).pow(2).sum()/len(X))
    RMSE.append(error)

RMSE = pd.DataFrame(RMSE) 
# RMSE.to_csv(r'C:\Users\l\Desktop\task_1a_1.csv', index = False, header=False)
print(RMSE)

          0
0  4.879264
1  4.879199
2  4.878675
3  4.882013
4  5.037022


#### Standardization + using cross_validate 

In [50]:
# scaling the inputs 
scaler = StandardScaler() 
scaled_X = scaler.fit_transform(X) 

#Standarize by above steps or 'normalize = False'?
penalty = [0.01,0.1,1,10,100]
RMSE = []

for lmd in penalty:
    model = Ridge(alpha = lmd, normalize = False, tol = 0.001, solver ='auto', random_state = 42) #solver?random_state?
    cv_results = cross_validate(model, scaled_X, y, cv=10,
                            n_jobs=-1,scoring=('neg_mean_squared_error'))
    result = np.sqrt(np.abs(cv_results["test_score"]).mean())
    RMSE.append(result)
    
RMSE = pd.DataFrame(RMSE) 
# RMSE.to_csv(r'C:\Users\l\Desktop\task_1a_2.csv', index = False, header=False)
print(RMSE)

[4.8785426102416105, 4.878477774804437, 4.877962725773613, 4.881405239155095, 5.037399687059372]


#### Standardization + using kf.split()

In [62]:
# report the average of the RMSEs on the left-out folds
scaler = StandardScaler() 
scaled_X = scaler.fit_transform(X) 
scaled_X = pd.DataFrame(scaled_X)

kf = KFold(n_splits=10, random_state = 42) # Define the split - into 10 folds 
kf.get_n_splits(scaled_X) # returns the number of splitting iterations in the cross-validator

penalty = [0.01,0.1,1,10,100]
RMSE = []

for lmd in penalty:
    error = 0
    model = Ridge(alpha=lmd, random_state = 42)
    for train_index, test_index in kf.split(scaled_X):
        X_train, X_test = scaled_X.iloc[train_index,:], scaled_X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test) 
        error += np.sqrt((y_pred - y_test).pow(2).mean())
    RMSE.append(error/10) 

RMSE = pd.DataFrame(RMSE) 
RMSE.to_csv(r'C:\Users\l\Desktop\task_1a_3.csv', index = False, header=False)
print(RMSE)

          0
0  4.796570
1  4.796415
2  4.795024
3  4.791487
4  4.924880


In [37]:
# get rid of the outliers (outside of 3*std)
X_selected = X[(np.abs(stats.zscore(X)) < 3).all(axis=1)]
y_selected = y[(np.abs(stats.zscore(X)) < 3).all(axis=1)]
scaler = StandardScaler() 
scaled_X = scaler.fit_transform(X_selected) 

#Standarize by above steps or 'normalize = False'?
penalty = [0.01,0.1,1,10,100]
RMSE = []

for lmd in penalty:
    model = Ridge(alpha = lmd, normalize = False, tol = 0.001, solver ='auto', random_state = 42) #solver?random_state?
#     cv_results = cross_validate(model, scaled_X, y, cv=10)
    predictions = cross_val_predict(model, scaled_X, y_selected, cv=10)
#     plt.scatter(y, predictions)
    error = np.sqrt((y_selected-predictions).pow(2).sum()/len(X_selected))
    RMSE.append(error)

RMSE = pd.DataFrame(RMSE) 
# RMSE.to_csv(r'C:\Users\l\Desktop\task_1a_1.csv', index = False, header=False)
print(RMSE)

          0
0  4.253636
1  4.253494
2  4.252214
3  4.248638
4  4.375949
